In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import spacy
import nltk
from spacy.lang.ru.examples import sentences
from nltk.corpus import stopwords
from multiprocessing import Pool
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
RAND = 10

In [10]:
df = pd.read_excel('dataset.xlsx')

In [11]:
df

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,749208109,Пусть это побудет здесь БорьбаВпрямомЭфире How...,esport
2,749208109,Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...,esport
3,749208109,Не знаю как вам но мне стилистика нравится пус...,esport
4,749208109,Скриншоты из новой главы. Тэхунчика показали и...,esport
...,...,...,...
53193,910636962,8 битная буря снова накрыла пикселями автомоби...,autosport
53194,669736851,Ира Сидоркова объясняет как сказалась на ее ма...,autosport
53195,558919241,24 я ракетка мира хорват Марин Чилич обыграл и...,tennis
53196,776944963,Стал известен календарь мужской сборной России...,volleyball


In [12]:
df = df.drop_duplicates()

In [13]:
X = df.drop(columns='category', axis=1)
y = df['category']

In [14]:
X_train_old, X_test_old, y_train_old, y_test_ol = train_test_split(X, y, test_size=0.2, random_state=RAND, shuffle=True, stratify=y)

Убедимся, что после разделения на тренировочные и тестовые данные классы распределены одинаково

In [ ]:
df['category'].value_counts() / df.shape[0] * 100 - y_train.value_counts() / y_train.shape[0] * 100

category
athletics      -0.000043
volleyball     -0.000516
tennis         -0.000041
winter_sport    0.000908
motosport      -0.000989
boardgames      0.000434
esport          0.000909
football        0.000436
martial_arts    0.000436
basketball     -0.000986
autosport       0.000446
extreme         0.000446
hockey         -0.001439
Name: count, dtype: float64

# Предобработка данных

In [ ]:
! python -m spacy download ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

In [ ]:
def cleaning(text):
    doc = nlp(text)
    ans = [token.lemma_.lower() for token in doc if token.lemma_.lower() not in stop_words and token.lemma_.lower().isalpha()]
    return ' '.join(ans)

In [ ]:
def cleaning(text):
    doc = nlp(text)
    ans = [token.lemma_.lower() for token in doc if token.lemma_.lower() not in stop_words and token.lemma_.lower().isalpha()]
    return ' '.join(ans)

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    results = []
    for i in tqdm(pool.imap(func, df_split), total=n_cores):
        results.append(i)

    pool.close()
    pool.join()
    return pd.concat(results)

def apply_cleaning(df):
    df['text'] = df['text'].apply(lambda x: cleaning(x))
    return df

In [ ]:
X_train_1 = X_train[:8500]
X_train_2 = X_train[8500:17000]
X_train_3 = X_train[17000:25500]
X_train_4 = X_train[25500:34000]
X_train_5 = X_train[34000:]

In [ ]:
X_train_1 = parallelize_dataframe(X_train_1, apply_cleaning)

100%|██████████| 4/4 [05:55<00:00, 88.83s/it] 


In [ ]:
X_train_2 = parallelize_dataframe(X_train_2, apply_cleaning)

100%|██████████| 4/4 [05:55<00:00, 88.81s/it] 


In [ ]:
X_train_3 = parallelize_dataframe(X_train_3, apply_cleaning)

100%|██████████| 4/4 [05:54<00:00, 88.63s/it] 


In [ ]:
X_train_4 = parallelize_dataframe(X_train_4, apply_cleaning)

100%|██████████| 4/4 [05:56<00:00, 89.08s/it]


In [ ]:
X_train_5 = parallelize_dataframe(X_train_5, apply_cleaning)

100%|██████████| 4/4 [05:37<00:00, 84.34s/it] 


In [ ]:
X_train = pd.concat([X_train_1, X_train_2, X_train_3, X_train_4, X_train_5])

In [ ]:
X_test = parallelize_dataframe(X_test, apply_cleaning)

100%|██████████| 4/4 [07:37<00:00, 114.28s/it]


In [ ]:
X_train.to_excel('train.xlsx')
X_train.to_excel('train_no_index.xlsx', index=False)
X_test.to_excel('test.xlsx')
X_test.to_excel('test_no_index.xlsx', index=False)
y_train.to_excel('train_labels.xlsx')
y_train.to_excel('train_labels_no_index.xlsx', index=False)
y_test.to_excel('test_labels.xlsx')
y_test.to_excel('test_labels_no_index.xlsx', index=False)

## Доп обработка

In [83]:
X_train = pd.read_excel('train_no_index.xlsx')
X_test = pd.read_excel('test_no_index.xlsx')
y_train = pd.read_excel('train_labels_no_index.xlsx')
y_test = pd.read_excel('test_labels_no_index.xlsx')

In [84]:
X_train.drop(columns=['oid'], axis=1, inplace=True)
X_test.drop(columns=['oid'], axis=1, inplace=True)

In [52]:
dict_labels = {'esport': 0, 'martial_arts': 1, 'tennis': 2, 'basketball': 3,
        'volleyball': 4, 'boardgames': 5, 'athletics': 6, 'winter_sport': 7,
        'football': 8, 'motosport': 9, 'autosport': 10, 'extreme': 11, 'hockey': 12}
def lab_enc(text, dictin):
  return dictin[text]

In [41]:
y_train['category'] = y_train['category'].apply(lambda x: lab_enc(x, dict_labels))

In [43]:
y_test['category'] = y_test['category'].apply(lambda x: lab_enc(x, dict_labels))

In [85]:
df = pd.concat([X_train, X_test])
labels = pd.concat([y_train, y_test])
df['class_name'] = labels
df['class'] = df['class_name'].apply(lambda x: lab_enc(x, dict_labels))

In [86]:
df[df.duplicated()].values

array([['друг кидать предложку свой фото видео тренировка соревнование посмотреть горазд наш дружина',
        'extreme', 11],
       ['atokenoid ltokenoid otokenoid анонс онлайн турнир скачать античит участие необходимый премиум турнир пройти октябрь мск мск check in максимальный количество участник регистрация tokentokenoid призовой фонд место место место',
        'esport', 0],
       ['кроссовками баскетбольный магазин ghetto basket shop консультация tokenoidtokenoid стрельский',
        'basketball', 3],
       ...,
       ['открыть предзаказ хребты безумие книга игра лавкрафтианским миф которой предстоять возглавить антарктический экспедиция которая столкнуться древний тайнам среди бескрайний холодный пустынь южный континент предстоять делать нелёгкий выбор выбирать безопасность любопытство внимательно следить происходить исход сюжет зависеть опасный событие древний артефакт загадочные смерть говорить антарктида бывать безумно интересный оформлять заказ ссылка предзаказ hotokenoi

In [89]:
df = df.reset_index().drop(columns=['index'], axis=1)

In [90]:
df

,text,class_name,class
0,уход ислам мамедов плотный удушающего реакция ...,martial_arts,1
1,ждать диллона прошлый неделя обещать конец нед...,basketball,3
2,энтони джошуа продолжать подготовка реванш уси...,martial_arts,1
3,мотогонка лёд speetokenoid tokenoid личный чем...,motosport,9
4,присылать ваш беговой фото публикация коммента...,athletics,6
...,...,...,...
52709,рафаэль надаль рассказать взаимоотношение фина...,tennis,2
52710,открыть предзаказ хребты безумие книга игра ла...,boardgames,5
52711,огромный успех эдди хирн цифра ppv канело биво...,martial_arts,1
52712,мочь хороший делать нравиться спорт сделать са...,athletics,6


In [93]:
df = df.drop_duplicates()
df = df.reset_index().drop(columns=['index'], axis=1)

In [94]:
df

,text,class_name,class
0,уход ислам мамедов плотный удушающего реакция ...,martial_arts,1
1,ждать диллона прошлый неделя обещать конец нед...,basketball,3
2,энтони джошуа продолжать подготовка реванш уси...,martial_arts,1
3,мотогонка лёд speetokenoid tokenoid личный чем...,motosport,9
4,присылать ваш беговой фото публикация коммента...,athletics,6
...,...,...,...
47950,карлос алькараз прокомментировать победа четве...,tennis,2
47951,мотоспорт wsbk porttokenoid гонка ltokenoid на...,motosport,9
47952,кокорин пропустит матч специей иза негодования...,esport,0
47953,рафаэль надаль рассказать взаимоотношение фина...,tennis,2


In [96]:
df.to_excel('data.xlsx', index=False)

In [97]:
from google.colab import files
files.download('data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>